# Fisher's exact P-value approach {#sec-fisher}

These are my notes from reading chapter 5 in @imbens2015causal.

Fisher's aim in his original work on experimentation was to asses the *sharp (or exact) null hypothesis* -- the hypothesis that a treatment had no effect whatsoever, meaning that the potential outcomes for being treated and not treated are the same for each unit in the data. (Compared to the null hypothesis that there is no treatment effect on average, which we use more frequently in experiments, Fisher's hypothesis is stricter -- of, if you like, sharper -- but the former is, arguably, of more practical interest in most cases.)

Unter that null hypothesis, the unobserved potential outcomes are known -- they are the same as the observed outcome -- and we can use the randomisation distribution to calculate P-values. The randomisation distribution is the distribution of values of the test statistic based on all possible assignment vectors. We can calculate P-values by calculating the test statistic for all possible assignments, and then calculate the probability that the test-statistic is as extreme or more extreme than the value of the test statistic in our experiment.

Three important points about the approach:

- The key point about the sharp null hypothesis is that it allows us to impute all unobserved potential outcomes, not that the treatment effect is zero. The postulated treatment effect could thus be any deterministic function of the available data (e.g. an additive or multiplicative homogenous effect or a function of observed covariates).

- The only source of randomness in this framework is the randomness introduced by randomisation, meaning there is no randomness from sampling because we are interested in conclusions about the specific sample at hand, not about a superpopulation.

- The approach is truly non-parametric in that it doesn't rely on a model of outcomes at all. In particular, we don't model the distribution of test statistic values but calculate it directly. We can do that because we know the assignment mechanism, which is known by definition in a completely randomised experiment, and because of the strict null hypothesis, which allows us to solve the fundamental problem of causal inference by imputing the unobserved potential values.

- @imbens2015causal point out that while Fisher's approach is often a good starting point, we are often concerned not only with learning that a treatment has some effect for some individual (which is what rejecting the sharp null hypothesis implies) but with estimating an average treatment effect without worrying about variations of the effect. For this, we need to rely on [Neyman's approach](neyman.qmd).


## A simple example

We have a sample of six units, three assigned to treatment ($W_i = 1$) and three to control ($W_i = 0$). The first two columns in the table show the (unobserved) potential outcomes under control and treatment status, the third and fourth column show the observed treatment status and outcome. For each of the six units the outcome value we observe is the potential outcome corresponding to the treatment status.

|   | $Y_i(0)$ | $Y_i(1)$ | $W_i$ | $Y_i^{obs}$ |
|---|----------|----------|-------|-------------|
| 1 | ?        | 3        | 1     | 3           |
| 2 | ?        | 5        | 1     | 5           |
| 3 | ?        | 0        | 1     | 0           |
| 4 | 4        | ?        | 0     | 4           |
| 5 | 0        | ?        | 0     | 0           |
| 6 | 1        | ?        | 0     | 1           |

This table highlights the fundamental problem of causal inference -- we can only ever observe one potential outcome for each unit.

However, Fisher's sharp null hypothesis asserts that:

$$
H_0: Y_i(1) = Y_i(0) \quad \text{for $i = 1, \dots, 6$},
$$

which makes filling in the missing values trivial:

|   | $Y_i(0)$ | $Y_i(1)$ | $W_i$ | $Y_i^{obs}$ |
|---|----------|----------|-------|-------------|
| 1 | (3)        | 3        | 1     | 3           |
| 2 | (5)        | 5        | 1     | 5           |
| 3 | (0)        | 0        | 1     | 0           |
| 4 | 4        | (4)        | 0     | 4           |
| 5 | 0        | (0)        | 0     | 0           |
| 6 | 1        | (1)        | 0     | 1           |

There are a number of different test statistics we could use (and @imbens2015causal discuss and compare an number of them). Following the book, for this example I use the absolute value of the difference in average outcome by treatment status, $T(W, Y^{obs}) = |\bar{Y}_t^{obs} - \bar{Y}_c^{obs}|$, which is a function of the random assignment $W$ and the observed values $Y^{obs}$.

For our little experiment above, we can calculate the test statistic easily as:

$$
\begin{aligned}
T^{obs} &= |(Y_1^{obs} + Y_2^{obs} + Y_3^{obs})/3 - (Y_4^{obs} + Y_5^{obs} + Y_6^{obs})/3| \\
&= |(3 + 5 + 0)/3 - (4 + 0 + 1)/3| \\
&= 8/3 - 5/3 \\
&= 1
\end{aligned}
$$

To calculate the P-value, we need the distribution of test statistics under all possible random assignments. There are $\begin{psmallmatrix}6\\3\end{psmallmatrix} = 20$ different assignments, and we can calculate the distribution using Python:


In [ ]:
import numpy as np
from itertools import combinations
import seaborn as sns

y = np.array([3, 5, 0, 4, 0, 1])
idx = range(len(y))

ts = []
for w in combinations(idx, 3):
    w0, w1 = list(set(idx) - set(w)), list(w)
    y0, y1 = y[w0], y[w1]
    t = abs(np.mean(y1) - np.mean(y0))
    ts.append(t)

sns.histplot(ts);

With that distribution in hand, we can easily calculate the P-value:


In [ ]:
p = np.mean([t >= 1 for t in ts])
p

The P-value indicates that if there is no treatment effect, we'd expect a value of the test statistic equal to 1 or even larger in 50 out of 100 random experiments, which does not provide any evidencen against the null hypothesis.

## Useful resources

@imbens2015causal